In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from typing import Any, Optional
import os
import gc
import datetime
import random
from time import time

import numpy as np
import pandas as pd
import datatable as dtable
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler

In [ ]:
train = dtable.fread('../input/jane-street-market-prediction/train.csv').to_pandas()
# train = train.query('date > 85').reset_index(drop = True)
train = train.query('weight != 249').reset_index(drop = True)
train = train.query('weight > 0').reset_index(drop = True)
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns}) #limit memory use
train.fillna(train.mean(),inplace=True)

features = [c for c in train.columns if 'feature' in c]
resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

In [ ]:
scaler = StandardScaler()
scaler.fit(train[features])
train.loc[:, features] = scaler.transform(train.loc[:, features])

## Calc IC and IR

In [ ]:
ic_list = []

for d, data in tqdm(train.groupby('date')):
    ic_list.append(data.corr()['resp'][features])

In [ ]:
ic_df = pd.DataFrame(ic_list, index=train['date'].unique())
ic_df

In [ ]:
for f in features[:10]:
    ic_df[f].plot(figsize=(16,9)).set_xlabel(f)
    plt.show()

In [ ]:
ir_df = ic_df.mean() / ic_df.std()

## Plot the IC and IR sort values

In [ ]:
ic_df.mean().sort_values(ascending=False).plot.bar(fontsize=10, figsize=(16,9))

In [ ]:
ir_df.sort_values(ascending=False).plot.bar(fontsize=10, figsize=(16,9))

## Plot the IC auto-correaltion

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf
for f in features[:10]:
    plot_acf(ic_df[f], alpha = 0.05, title = f)
    plt.show()

# Use only Feature_0 negative part

In [ ]:
f0n_ic_list = []

for d, data in tqdm(train.query('feature_0 < 0').groupby('date')):
    f0n_ic_list.append(data.corr()['resp'][features])

In [ ]:
f0n_ic_df = pd.DataFrame(f0n_ic_list, index=train['date'].unique())
f0n_ic_df

In [ ]:
f0n_ir_df = f0n_ic_df.mean() / f0n_ic_df.std()

In [ ]:
for f in features[:10]:
    f0n_ic_df[f].plot(figsize=(16,9)).set_xlabel(f)
    plt.show()

In [ ]:
f0n_ic_df.mean().sort_values(ascending=False).plot.bar(fontsize=10, figsize=(16,9))

In [ ]:
f0n_ir_df.sort_values(ascending=False).plot.bar(fontsize=10, figsize=(16,9))

feature_0 negative part has more correlation with resp

**Conclusion: Although the average correlation of features to resp is less than 5%, the intraday correlation is cyclical and someday can reach 20%-40%, if only use the feature_0 negative part, correlation can reach 60%+. This means that if only use this feature high correlation part to make a prediction, the accuracy will be quite high. Maybe we can use this information to build a better feature selection strategy.**